In [ ]:
import json
%load_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd
import pickle
import json

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer

#from rulefit import RuleFit
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import BayesianRidge, LinearRegression, LogisticRegression, ElasticNet
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, TunedThresholdClassifierCV
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
#from category_encoders import JamesSteinEncoder, CatBoostEncoder
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.feature_selection import SelectFromModel, VarianceThreshold

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

# add ../src to python path
sys.path.insert(0, os.path.join(os.path.abspath('.'),'..', 'src'))

import tree_utils

In [ ]:
# Try from sklego.meta import HierarchicalClassifier
# hc = HierarchicalClassifier(
#    estimator=LogisticRegression(),
#    groups=groups
#).fit(X, y)
#hc.estimators_

In [ ]:
data_path =  r"J:\Onderzoek\21-763_rvanes_MiniECG-2-Data\E_ResearchData\2_ResearchData\Analysis"
file_name =  r"input_decision_tree_complete.pkl"

num_splits = 10
num_repeats = 10
MAX_FEATURES = 100 # only relevant if USE_LEAD_COMBOS is True
USE_CLASS_WEIGHT = False
ALL_FEATURES = True
USE_REDUCED_LABELS = False
USE_CALIBRATED_CLASSIFIER = True
USE_LEAD_COMBOS = False
COMBO_WITH_FEATURE_FILTER = False

CatEncoderType = 'onehot' # 'onehot' or 'ordinal'

ALL_FEATURES_STRING = "_useAllvars" if ALL_FEATURES else ""
CLASS_WEIGHT_STRING = "_withClassWeights" if USE_CLASS_WEIGHT else ""
REDUCED_LABEL_STRING = "_withReducedLabels" if USE_REDUCED_LABELS else ""
CALIBRATED_CLASSIFIER_STRING = "_withCalibratedClassifier" if USE_CALIBRATED_CLASSIFIER else ""
FEATURE_COMBO_STRING = "_withLeadCombos" if USE_LEAD_COMBOS else ""

In [ ]:
with open(os.path.join(data_path, file_name), 'rb') as f:
    input_decision_tree_complete = pickle.load(f)

In [ ]:
DATA = pd.DataFrame(input_decision_tree_complete).T

In [ ]:
out_folder =r"J:\Onderzoek\21-763_rvanes_MiniECG-2-Data\E_ResearchData\2_ResearchData\Parquet"
DATA.to_parquet(os.path.join(out_folder,'DATA.parquet'))

In [ ]:
morphology_columns = [c for c in DATA.columns if 'morphology' in c]
lead_columns = [c for c in DATA.columns if ('lead' in c) & ('morphology' not in c)]

for c in morphology_columns:
    DATA.loc[:, c] = DATA[c].apply(lambda x: x[0].strip(",").strip(" "))
    DATA.loc[:, c] = DATA[c].apply(lambda x: x if x.strip()!="" else "none")

In [ ]:
morphology_values = []
for c in morphology_columns:
    morphology_values.extend(DATA[c].unique().tolist())
morphology_values = list(set(morphology_values))

In [ ]:
ComboFunctions = {
    'AbsFactor' : lambda x,y: abs(x-y) # np.sign(x*y)*np.sqrt(np.abs(x*y))
}

In [ ]:
if USE_LEAD_COMBOS:
    Enriched_DF, new_columns = tree_utils.create_feature_combinations(df=DATA[lead_columns], 
                                           lambda_functions=ComboFunctions)
    DATA = DATA.drop(lead_columns, axis=1)
    DATA = pd.concat([DATA, Enriched_DF], axis=1)


# Model prepping

In [ ]:
# TODO: need to add a feature combiner, perhaps use PySR or GpLearn, focus on vectors

impute_kwargs = {
    'estimator': LinearRegression(), 
    'random_state':7,
    'imputation_order': 'ascending', 
    'skip_complete': True,
    'max_iter': 2000,
    'initial_strategy': 'median',
    'add_indicator': True
}
gradientboosting_kwargs = {
    'n_estimators': 500, 
    'max_depth': 10, 
    'learning_rate':0.01,
    'max_leaf_nodes':40,
    'random_state': 7
}
randomforest_kwargs = {
    'n_estimators': 150,
    'max_depth': 6,
    'min_samples_split': 10, 
    'min_samples_leaf': 10,
    'random_state': 7
}
rulefit_kwargs={
    'tree_size': 10,
    'max_rules': 100,
    'tree_generator': GradientBoostingClassifier(**gradientboosting_kwargs)
}
decisiontree_kwargs = {
    'criterion':'gini', 
    'splitter':'best', 
    'max_depth':5, 
    'min_samples_split':10, 
    'min_samples_leaf': 5, 
    'min_weight_fraction_leaf':0.05, 
    'max_features':None, 
    'random_state':7, 
    'max_leaf_nodes':50,
    'class_weight': 'balanced'
}
xgboost_kwargs = {
    'n_estimators': 150,
    'max_depth': 6,
    'max_leaves': 50,
    'learning_rate': 2e-3,
    'gamma': 0.4,
    'subsample': 0.55,
    'colsample_bytree':0.85,
    'reg_alpha': 0.005
}
logistic_kwargs = {
    'penalty': 'elasticnet', 
    'dual': False, 
    'tol': 0.0001, 
    'C':1.0, 
    'fit_intercept': True, 
    'intercept_scaling':1, 
    'class_weight':None, 
    'random_state':7, 
    'solver': 'saga', 
    'max_iter':2000, 
    'verbose': 0, 
    'warm_start': False, 
    'n_jobs':-1, 
    'l1_ratio':0.5
}
calibration_kwargs ={
    'method': 'sigmoid',
    'n_jobs': -1, 
    'ensemble': True,
    'cv': 5
}
ordinal_encoder_kwargs = {
    'categories': 'auto',
    'dtype': int,
    'handle_unknown':'use_encoded_value',
    'unknown_value': -2,
    'encoded_missing_value': -1
}
onehot_kwargs = {
    'drop': 'first', 
    'sparse_output': False,
    'min_frequency': 0.15,
    'handle_unknown': 'ignore'
}

combine_kwargs = {}
combine_kwargs['impute'] = impute_kwargs.copy()
combine_kwargs['impute']['estimator'] = str(type(combine_kwargs['impute']['estimator']))
combine_kwargs['decisiontree'] = decisiontree_kwargs
combine_kwargs['xgboost'] = xgboost_kwargs
combine_kwargs['logistic'] = logistic_kwargs
combine_kwargs['calibration'] = calibration_kwargs

if CatEncoderType == 'onehot':
    combine_kwargs['catencoder'] = onehot_kwargs.copy()
    combine_kwargs['catencoder']['type'] = 'onehot'
elif CatEncoderType == 'ordinal':
    combine_kwargs['catencoder'] = ordinal_encoder_kwargs.copy()
    combine_kwargs['catencoder']['type'] = 'ordinal'
else:
    raise ValueError("Invalid CatEncoderType")

In [ ]:
OrdEncoder = OrdinalEncoder(**ordinal_encoder_kwargs)
OneHot = OneHotEncoder(**onehot_kwargs)

if CatEncoderType == 'onehot':
    _catencoder = OneHot
else:
    _catencoder = OrdEncoder


PipeOrdEncoder = ColumnTransformer([("cat_encoder", _catencoder, morphology_columns)],
                                       remainder='passthrough')

FFilter = SelectFromModel(estimator=LogisticRegression(**logistic_kwargs),
                            max_features=MAX_FEATURES)

_imputer = IterativeImputer(**impute_kwargs)

if ALL_FEATURES:
    _cat_enc = PipeOrdEncoder
else:
    _cat_enc = None

if (USE_LEAD_COMBOS) & (COMBO_WITH_FEATURE_FILTER):
    _feature_filter = FFilter
    _imputer = SimpleImputer(strategy='median', add_indicator=True)
else:
    _feature_filter = VarianceThreshold()

if USE_CALIBRATED_CLASSIFIER:
    RF_clf = CalibratedClassifierCV(RandomForestClassifier(**randomforest_kwargs), 
                                    **calibration_kwargs)
    GBC_clf = CalibratedClassifierCV(GradientBoostingClassifier(**gradientboosting_kwargs),
                                     **calibration_kwargs)
    XGB_clf = CalibratedClassifierCV(XGBClassifier(**xgboost_kwargs),
                                     **calibration_kwargs)
    DT_clf = CalibratedClassifierCV(DecisionTreeClassifier(**decisiontree_kwargs),
                                     **calibration_kwargs)
    LR_clf = CalibratedClassifierCV(LogisticRegression(**logistic_kwargs),
                                     **calibration_kwargs)
else:
    RF_clf = RandomForestClassifier(**randomforest_kwargs)
    GBC_clf = GradientBoostingClassifier(**gradientboosting_kwargs)
    XGB_clf = tree_utils.GenericCalibratedClassifier(XGBClassifier(**xgboost_kwargs))
    DT_clf = DecisionTreeClassifier(**decisiontree_kwargs)
    LR_clf = LogisticRegression(**logistic_kwargs)

le_pipe_rf = Pipeline([
    ("CatEncoder", _cat_enc),
    ("Impute", _imputer),
    ("FeatureFilter", _feature_filter),
    ("RandomForest", RF_clf)])

le_pipe_gbc = Pipeline([
    ("CatEncoder", _cat_enc),
    ("Impute", _imputer),
    ("FeatureFilter", _feature_filter),
    ("GradientBoosting", GBC_clf)])

le_pipe_xgb = Pipeline([
    ("CatEncoder", _cat_enc),
    ("Impute", _imputer),
    ("FeatureFilter", _feature_filter),
    ("XGBoost", XGB_clf)])

le_pipe_dt = Pipeline([
    ("CatEncoder", _cat_enc),
    ("Impute", _imputer),
    ("FeatureFilter", _feature_filter),
    ("DecisionTree", DT_clf)])

le_pipe_lr = Pipeline([
    ("CatEncoder", _cat_enc),
    ("Impute", _imputer),
    ("FeatureFilter", _feature_filter),
    ("LogisticRegression", LR_clf)])

PipeDict = {}
#PipeDict['rf'] = le_pipe_rf
#PipeDict['gbc'] = le_pipe_gbc
PipeDict['xgb'] = le_pipe_xgb
PipeDict['dt'] = le_pipe_dt
PipeDict['lr'] = le_pipe_lr


# Axis model

In [ ]:
target_col = "Heart Axis Diagnosis"
if not USE_REDUCED_LABELS:
    target_inclusion = ['Left', 'Normal', 'Right']
else:
    target_inclusion = ['Left', 'Normal', 'Right', 'Extreme']
    
Reduction_map = {'Left': 'Disease', 
                 'Right': 'Disease',
                 'Extreme': 'Disease',
                 'Normal': 'Normal'}
if ALL_FEATURES:
    features_to_use = []
else:
    features_to_use = ['qrs_vector mean lead_0',
                     'p_vector mean lead_0',
                     't_vector mean lead_0',
                     'qrs_vector mean lead_1',
                     'p_vector mean lead_1',
                     't_vector mean lead_1',
                     'qrs_vector mean lead_2',
                     'p_vector mean lead_2',
                     't_vector mean lead_2',
                     'qrs_vector mean lead_3',
                     'p_vector mean lead_3',
                     't_vector mean lead_3',
                     'qrs_vector mean lead_4',
                     'p_vector mean lead_4',
                     't_vector mean lead_4',
                     'qrs_vector mean lead_5',
                     'p_vector mean lead_5',
                     't_vector mean lead_5',
                     'qrs_vector mean lead_6',
                     'p_vector mean lead_6',
                     't_vector mean lead_6',
                     'qrs_vector mean lead_7',
                     'p_vector mean lead_7',
                     't_vector mean lead_7'
                       ]

In [ ]:
if len(features_to_use)==0:
    meas_cols = [c for c in DATA.columns if ('Dataset' not in c) 
                 & (target_col not in c)
                 & ('Diagnosis' not in c)]
else:
    meas_cols = features_to_use
    
fstring = f"AXIS_{CLASS_WEIGHT_STRING}{ALL_FEATURES_STRING}{REDUCED_LABEL_STRING}{CALIBRATED_CLASSIFIER_STRING}{FEATURE_COMBO_STRING}_CatEnc{CatEncoderType}"
os.makedirs(os.path.join(data_path, fstring), exist_ok=True)

AXIS_DATA = DATA.loc[DATA[target_col].isin(target_inclusion), meas_cols+[target_col]+['Dataset']]
if USE_REDUCED_LABELS:
    AXIS_DATA.loc[:, target_col] = AXIS_DATA[target_col].map(Reduction_map)
    
AXIS_DATA.to_parquet(os.path.join(data_path, fstring, 'DATA.parquet'))
AXIS_DATA = AXIS_DATA.drop('Dataset', axis=1)

json.dump(combine_kwargs, open(os.path.join(data_path, fstring, 'settings.json'), 'w'))


### TRAINING LOOP

In [ ]:
splitter = RepeatedStratifiedKFold(n_splits=num_splits, n_repeats=num_repeats, random_state=7)

X = AXIS_DATA.iloc[:, :-1]
Y = AXIS_DATA.iloc[:,-1]

lb = LabelBinarizer()
lbe = LabelEncoder()

 #(RES_LIST_AXIS[0]['Y_test'])
Yenc = lbe.fit_transform(Y.values) #(RES_LIST_AXIS[0]['Y_test'])
y_bin = lb.fit(Yenc)
ClassMap_AXIS = {i:c for i,c in enumerate(lbe.classes_)}

In [ ]:
ClassMap_AXIS

In [ ]:
RES_LIST_AXIS, RES_AXIS_DF, RES_AXIS_INDCS_DF = tree_utils.training_loop(X, Yenc, splitter, 
                              PipeDict, 
                              use_class_weights=USE_CLASS_WEIGHT, ClassMap=ClassMap_AXIS,
                              num_splits=num_splits, num_repeats=num_repeats, make_df=True)

### make roc and precision recall curves

In [ ]:
Y.value_counts()

In [ ]:
n_classes = len(lb.classes_)
colors = ['blue', 'green', 'red', 'yellow', 'magenta', 'cyan', 'black']

In [ ]:
PLOTS_AXIS = tree_utils.make_plots(RES_LIST_AXIS, lb,  n_classes, 
                                   colors, ClassMap_AXIS,
                                   output_map=os.path.join(data_path, fstring),
                                   show_plot=False, 
                                   models = list(PipeDict.keys()),
                                   plot_title="Heart Axis")
perf_list = tree_utils.get_performance(RES_LIST_AXIS, 
                                       threshold=1/n_classes,
                                       ClassMap=ClassMap_AXIS, 
                                       models = list(PipeDict.keys()),
                                       binarizer=lb)
PERF_AXIS = pd.DataFrame(perf_list)

In [ ]:
PERF_AXIS[['f1', 'precision', 'recall', 'specificity', 'model', 'Class']].groupby(['model', 'Class']).mean()

In [ ]:
tree_utils.net_benefit_curve_plot(RES_AXIS_DF, true_col_prefix='Y_test',
                                     pred_col_prefix='Y_pred',
                                     output_path=os.path.join(data_path, fstring),
                                     threshold_steps=20, 
                                     xlim=[0,0.5],
                                     ylim=[-1,1],
                                     plot_title="Heart Axis")

tree_utils.calibration_curve_plot(RES_AXIS_DF,
                                   true_col_prefix='Y_test',
                                   pred_col_prefix='Y_pred',
                                   output_path=os.path.join(data_path, fstring),
                                   n_bins=10,
                                   plot_title="Heart Axis",
                                   show_plot=True)

In [ ]:
RES_AXIS_DF.to_parquet(os.path.join(data_path, fstring, "RESULTS.parquet"))
RES_AXIS_INDCS_DF.to_parquet(os.path.join(data_path, fstring, "RESULTS_w_indices.parquet"))

# Muscle model

In [ ]:
target_col = "Diagnosis"
target_inclusion = ['SR','LVH','Microvoltages']
Reduction_map = {'Microvoltages': 'Disease', 
                 'LVH': 'Disease',
                 'SR': 'Normal'}
if ALL_FEATURES:
    features_to_use = []
else:
    features_to_use = ['qrs_vector mean lead_0',
                     'qrs_ampl mean lead_0',
                     'qrs_vector mean lead_1',
                     'qrs_ampl mean lead_1',
                     'qrs_vector mean lead_2',
                     'qrs_ampl mean lead_2',
                     'qrs_vector mean lead_3',
                     'qrs_ampl mean lead_3',
                     'qrs_vector mean lead_4',
                     'qrs_ampl mean lead_4',
                     'qrs_vector mean lead_5',
                     'qrs_ampl mean lead_5',
                     'qrs_vector mean lead_6',
                     'qrs_ampl mean lead_6',
                     'qrs_vector mean lead_7',
                     'qrs_ampl mean lead_7',
                     'morphology lead_0',
                     'morphology lead_1',
                     'morphology lead_2',
                     'morphology lead_3',
                     'morphology lead_4',
                     'morphology lead_5',
                     'morphology lead_6',
                     'morphology lead_7']

In [ ]:
_cat_enc = PipeOrdEncoder
for k in PipeDict.keys():
    PipeDict[k].set_params(CatEncoder= _cat_enc)

In [ ]:
if len(features_to_use)==0:
    meas_cols = [c for c in DATA.columns if ('Dataset' not in c) 
                 & (target_col not in c)
                 & ("Heart Axis Diagnosis" not in c)]
else:
    meas_cols = features_to_use
    
fstring = f"MUSCLE_{CLASS_WEIGHT_STRING}{ALL_FEATURES_STRING}{REDUCED_LABEL_STRING}{CALIBRATED_CLASSIFIER_STRING}{FEATURE_COMBO_STRING}_CatEnc{CatEncoderType}"
os.makedirs(os.path.join(data_path, fstring), exist_ok=True)

MUSCLE_DATA = DATA.loc[DATA[target_col].apply(lambda x: any([c in x for c in target_inclusion])), 
                       meas_cols+[target_col]+['Dataset']]

MUSCLE_DATA = MUSCLE_DATA.assign(Diagnosis=MUSCLE_DATA.Diagnosis.map({
                                                            'SR': 'SR',
                                                            'Microvoltages': 'Microvoltages',
                                                            'LVH': 'LVH',
                                                            'LAFB , LVH': 'LVH',
                                                            'Microvoltages , BF': 'Microvoltages',
                                                            'Microvoltages , RBBB': 'Microvoltages',
                                                            'Microvoltages , LAFB': 'Microvoltages',
                                                            'LVH , BF': 'LVH',
                                                            'LVH , RBBB': 'LVH',
                                                            'LVH , LBBB': 'LVH'
                                                        }))

if USE_REDUCED_LABELS:
    MUSCLE_DATA.loc[:, target_col] = MUSCLE_DATA[target_col].map(Reduction_map)
    
MUSCLE_DATA.to_parquet(os.path.join(data_path, fstring, f'DATA.parquet'))
MUSCLE_DATA = MUSCLE_DATA.drop('Dataset', axis=1)

json.dump(combine_kwargs, open(os.path.join(data_path, fstring, 'settings.json'), 'w'))


## Training loop

In [ ]:
splitter = RepeatedStratifiedKFold(n_splits=num_splits, n_repeats=num_repeats, random_state=7)
X = MUSCLE_DATA.iloc[:, :-1]
Y = MUSCLE_DATA.iloc[:,-1]

lb = LabelBinarizer()
lbe = LabelEncoder()
 #(RES_LIST_AXIS[0]['Y_test'])
Yenc = lbe.fit_transform(Y.values) #(RES_LIST_AXIS[0]['Y_test'])
lb.fit(Yenc)
ClassMap_MUSCLE = {i:c for i,c in enumerate(lbe.classes_)}

In [ ]:
ClassMap_MUSCLE

In [ ]:
RES_LIST_MUSCLE, RES_MUSCLE_DF, RES_MUSCLE_INDCS_DF =\
    tree_utils.training_loop(X, Yenc, splitter, PipeDict,  
                             use_class_weights=USE_CLASS_WEIGHT, 
                             ClassMap=ClassMap_MUSCLE, 
                             num_splits=num_splits, 
                             num_repeats=num_repeats, 
                             make_df=True)

### make roc and precision recall curves

In [ ]:
Y.value_counts()

In [ ]:
n_classes = len(lb.classes_)
colors = ['blue', 'green', 'red', 'yellow', 'magenta', 'cyan', 'black']

In [ ]:
PLOTS_MUSCLE = tree_utils.make_plots(RES_LIST_MUSCLE, lb,  n_classes, colors,
                                     ClassMap_MUSCLE,
                                     output_map=os.path.join(data_path, fstring),
                                     show_plot=False,
                                     models = list(PipeDict.keys()),
                                     plot_title="Heart Muscle")

perf_list = tree_utils.get_performance(RES_LIST_MUSCLE, 
                                       threshold=1/n_classes, 
                                       ClassMap=ClassMap_MUSCLE, 
                                       models = list(PipeDict.keys()),
                                       binarizer=lb)
PERF_MUSCLE = pd.DataFrame(perf_list)

In [ ]:
PERF_MUSCLE[['f1', 'precision', 'recall', 'specificity', 'model', 'Class']].groupby(['model', 'Class']).mean()

In [ ]:
tree_utils.net_benefit_curve_plot(RES_MUSCLE_DF, true_col_prefix='Y_test',
                                     pred_col_prefix='Y_pred',
                                     output_path=os.path.join(data_path, fstring),
                                     threshold_steps=20, 
                                     xlim=[0,0.5],
                                     ylim=[-1,1],
                                     plot_title="Heart Muscle")

tree_utils.calibration_curve_plot(RES_MUSCLE_DF,
                                   true_col_prefix='Y_test',
                                   pred_col_prefix='Y_pred',
                                   output_path=os.path.join(data_path, fstring),
                                   n_bins=10,
                                   plot_title="Heart Muscle",
                                   show_plot=True)

In [ ]:
RES_MUSCLE_DF.to_parquet(os.path.join(data_path, fstring, "RESULTS.parquet"))
RES_MUSCLE_INDCS_DF.to_parquet(os.path.join(data_path, fstring, "RESULTS_w_indices.parquet"))

# Conduction model

In [ ]:
target_col = "Diagnosis"
target_inclusion = ['BF', 'LBBB','RBBB','LAFB', 'SR']
Reduction_map = {'BF': 'Disease', 
                 'LBBB': 'Disease', 
                 'RBBB': 'Disease',
                 'LAFB': 'Disease',
                 'SR': 'Normal'}
features_to_use = []
_cat_enc = PipeOrdEncoder
for k in PipeDict.keys():
    PipeDict[k].set_params(CatEncoder= _cat_enc)

In [ ]:
if len(features_to_use)==0:
    meas_cols = [c for c in DATA.columns if ('Dataset' not in c) 
                 & (target_col not in c)
                 & ("Heart Axis Diagnosis" not in c)]
else:
    meas_cols = features_to_use
    
fstring = f"CONDUCTION{CLASS_WEIGHT_STRING}{ALL_FEATURES_STRING}{REDUCED_LABEL_STRING}{CALIBRATED_CLASSIFIER_STRING}{FEATURE_COMBO_STRING}_CatEnc{CatEncoderType}"
os.makedirs(os.path.join(data_path, fstring), exist_ok=True)

CONDUCTION_DATA = DATA.loc[DATA[target_col].apply(lambda x: any([c in x for c in target_inclusion])),  meas_cols+[target_col]+['Dataset']]

CONDUCTION_DATA = CONDUCTION_DATA.assign(Diagnosis=CONDUCTION_DATA.Diagnosis.map({
                                                                'SR': 'SR',
                                                                'BF': 'BF',
                                                                'RBBB': 'RBBB',
                                                                'LBBB': 'LBBB',
                                                                'LAFB': 'LAFB',
                                                                'LAFB , LVH': 'LAFB',
                                                                'Microvoltages , BF': 'BF',
                                                                'Microvoltages , RBBB': 'RBBB',
                                                                'Microvoltages , LAFB': 'LAFB', 
                                                                'LVH , BF': 'BF',
                                                                'LVH , RBBB': 'RBBB',
                                                                'LVH , LBBB': 'LBBB'
                                                            }))
if USE_REDUCED_LABELS:
    CONDUCTION_DATA.loc[:, target_col] = CONDUCTION_DATA[target_col].map(Reduction_map)
    
CONDUCTION_DATA.to_parquet(os.path.join(data_path, fstring, 'CONDUCTION.parquet'))
CONDUCTION_DATA = CONDUCTION_DATA.drop('Dataset', axis=1)

json.dump(combine_kwargs, open(os.path.join(data_path, fstring, 'settings.json'), 'w'))


## Training loop

In [ ]:
splitter = RepeatedStratifiedKFold(n_splits=num_splits, n_repeats=num_repeats, random_state=7)
X = CONDUCTION_DATA.iloc[:, :-1]
Y = CONDUCTION_DATA.iloc[:,-1]

lb = LabelBinarizer()
lbe = LabelEncoder()
Yenc = lbe.fit_transform(Y)
lb.fit(Yenc)    
ClassMap_CONDUCTION = {i:c for i,c in enumerate(lbe.classes_)}

In [ ]:
ClassMap_CONDUCTION

In [ ]:
RES_LIST_CONDUCTION, RES_CONDUCTION_DF, RES_CONDUCTION_INDCS_DF = tree_utils.training_loop(X, Yenc, splitter, PipeDict,
                                    use_class_weights=USE_CLASS_WEIGHT, 
                                    ClassMap=ClassMap_CONDUCTION,
                                    num_splits=num_splits,num_repeats=num_repeats,
                                    make_df=True)

### make roc and precision recall curves

In [ ]:
Y.value_counts()

In [ ]:
n_classes = len(lb.classes_)
colors = ['blue', 'green', 'red', 'yellow', 'magenta', 'cyan', 'black']

In [ ]:
PLOTS_CONDUCTION = tree_utils.make_plots(RES_LIST_CONDUCTION, lb,  n_classes, colors,
                                         ClassMap_CONDUCTION,
                                         output_map=os.path.join(data_path, fstring),
                                         show_plot=False,
                                         models = list(PipeDict.keys()),
                                         plot_title="Heart Conduction")

perf_list = tree_utils.get_performance(RES_LIST_CONDUCTION,
                                       threshold=1/n_classes, 
                                       ClassMap=ClassMap_CONDUCTION,
                                       models = list(PipeDict.keys()),
                                       binarizer=lb)

PERF_CONDUCTION = pd.DataFrame(perf_list)

In [ ]:
PERF_CONDUCTION[['f1', 'precision', 'recall', 'specificity', 'model', 'Class']].groupby(['model', 'Class']).mean()

In [ ]:
tree_utils.net_benefit_curve_plot(RES_CONDUCTION_DF, true_col_prefix='Y_test',
                                     pred_col_prefix='Y_pred',
                                     output_path=os.path.join(data_path, fstring),
                                     threshold_steps=20, 
                                     xlim=[0,0.5],
                                     ylim=[-1,1],
                                     plot_title="Heart Conduction")

tree_utils.calibration_curve_plot(RES_CONDUCTION_DF,
                                   true_col_prefix='Y_test',
                                   pred_col_prefix='Y_pred',
                                   output_path=os.path.join(data_path, fstring),
                                   n_bins=10,
                                   plot_title="Heart Conduction",
                                   show_plot=True)

In [ ]:
RES_CONDUCTION_DF.to_parquet(os.path.join(data_path, fstring, "RESULTS.parquet"))
RES_CONDUCTION_INDCS_DF.to_parquet(os.path.join(data_path, fstring, "RESULTS_w_indices.parquet"))